In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.io
import time
import glob,os
import cv2

In [6]:
# define cif file position
file = '/srv/home/chenyu/cif/Al2O3_slab.xyz'

# define target model x,y,z size in A, better use size_x = size_y for multislice model
repeat_z = 250
# define unit cell size along z direction
z_uc = 4.762

# padding in A for x,y,z direction. Notice that pad_z will be useless in Prismatic
pad_x = 0.1
pad_y = 0.1
pad_z = 0.0

file_mul = '/srv/home/chenyu/cif/Al2O3_100nm_multislice.xyz'
file_vesta = '/srv/home/chenyu/cif/Al2O3_100nm.xyz'

# build dictionary for Debye-Waller factor, make sure key names contain all possible atom types
DWdict = {
    "Al":"0.053",
    "O":"0.059"
}
Zdict = {
    "Al":"13",
    "O":"8"
}

# Header line at beginning
header = 'Al2O3 model for 100nm'

In [7]:
# Extract lattice parameters
f_write = open(file_mul,'w')
f_write_2 = open(file_vesta,'w')
atom_num = 0
max_x = 0
max_y = 0
max_z = 0
with open(file) as f:
    datafile = f.readlines()
    for ilayer in range(repeat_z):
        # loop over all atom coordiantes, starts from line 3
        for i in range(2,len(datafile)):
            temp = datafile[i].split(' ')
            # should have a better way to extract x,y,z from each line!
            # use a while loop to remove all spaces in the line
            i=0
            while len(temp)!=4:
                if len(temp[i])==0:
                    del temp[i]
                else: 
                    i=i+1
            # exchange x and z
            z = float(temp[1]) + pad_z + ilayer * z_uc
            y = float(temp[2]) + pad_y
            x = float(temp[3][:-1]) + pad_x
            atom_type = temp[0]

            if x >= 0 and y >= 0 and z >= 0:
                atom_num = atom_num + 1
                if x + pad_x > max_x:
                    max_x = x + pad_x
                if y + pad_y > max_y:
                    max_y = y + pad_y
                if z + pad_z > max_z:
                    max_z = z + pad_z
                # generate multislice file line, occupancy is 1 by default
                content = str(Zdict[atom_type]) + ' '   # Z number
                content = content + ("{:.3f}".format(x + pad_x) + ' ')   # x cor
                content = content + ("{:.3f}".format(y + pad_y) + ' ')   # y cor
                content = content + ("{:.3f}".format(z + pad_z) + ' 1 ')   # z cor and occupancy, default as 1
                content = content + (DWdict[atom_type] + '\n')   # D-W factor
                f_write.write(content)

                # generate vesta file line
                content = atom_type + ' '   # Z number
                content = content + ("{:.3f}".format(x + pad_x) + ' ')   # x cor
                content = content + ("{:.3f}".format(y + pad_y) + ' ')   # y cor
                content = content + ("{:.3f}".format(z + pad_z) + '\n')   # z cor
                f_write_2.write(content)
            
f_write.write('-1')  # write -1 at the end of file
f_write.close()
f_write_2.close()

# Add top two rows to multislice file, header and total size
f_write = open(file_mul,'r')
datafile = f_write.readlines()
content = "{:.3f}".format(max_x) + ' ' + "{:.3f}".format(max_y) + ' ' + "{:.3f}".format(max_z) + '\n'
datafile.insert(0,content)
content = 'Total ' + str(atom_num) + ' atoms.' + header + '\n'
datafile.insert(0,content)
f_write.close()
f_write = open(file_mul,'w')
f_write.writelines(datafile)
f_write.close()

# Add top two rows to vesta file, total atom number and header
f_write = open(file_vesta,'r')
datafile = f_write.readlines()
content = header + '\n'
datafile.insert(0,content)
content = str(atom_num) + '\n'
datafile.insert(0,content)
f_write.close()
f_write = open(file_vesta,'w')
f_write.writelines(datafile)
f_write.close()